# 📊 Análise de Evasão de Clientes (Churn) - Telecom X

Notebook simples e direto ao ponto: extração, limpeza, features, EDA, correlação e conclusões.

## 1) Configuração e Importação

In [ ]:
import pandas as pd
import requests
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style='whitegrid')
plt.rcParams['figure.dpi'] = 100
plt.rcParams['figure.figsize'] = (10, 6)

url = 'https://raw.githubusercontent.com/ingridcristh/challenge2-data-science/main/TelecomX_Data.json'
resp = requests.get(url, timeout=30)
resp.raise_for_status()
data = resp.json()
df = pd.DataFrame(data)
df.head()


## 2) Limpeza e Pré-processamento

In [ ]:
# Desaninhamento de colunas
for nested in ['customer', 'phone', 'internet', 'account']:
    if nested in df.columns:
        norm = pd.json_normalize(df[nested])
        df = pd.concat([df.drop(columns=[nested]), norm], axis=1)

# Função utilitária para localizar colunas por palavras-chave
def find_col(cols, keywords):
    for c in cols:
        cl = c.lower()
        if all(k in cl for k in keywords):
            return c
    return None

col_total = find_col(df.columns, ['total','charge'])
col_monthly = find_col(df.columns, ['monthly','charge'])
col_tenure = find_col(df.columns, ['tenure'])
col_churn = find_col(df.columns, ['churn'])

# Tipos e valores ausentes
if col_total:
    df[col_total] = df[col_total].replace(' ', pd.NA)
    df[col_total] = pd.to_numeric(df[col_total], errors='coerce').fillna(0)
if col_monthly:
    df[col_monthly] = pd.to_numeric(df[col_monthly], errors='coerce')
if col_tenure:
    df[col_tenure] = pd.to_numeric(df[col_tenure], errors='coerce')
if col_churn:
    df[col_churn] = df[col_churn].fillna('No')

# Padronizar categorias de serviços
for cand in ['OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','StreamingTV','StreamingMovies','MultipleLines','InternetService','PhoneService']:
    col = find_col(df.columns, [cand.lower()])
    if col and df[col].dtype == 'object':
        df[col] = df[col].replace({'No internet service':'No','No phone service':'No'})

df.info()


## 3) Engenharia de Features e Padronização

In [ ]:
# Contas_Diarias
if col_monthly:
    df['Contas_Diarias'] = (df[col_monthly] / 30).round(2)
else:
    df['Contas_Diarias'] = 0

# Renomear colunas para PT
rename_map = {
    'customerID':'ID_Cliente','gender':'Genero','SeniorCitizen':'Idoso','Partner':'Parceiro','Dependents':'Dependentes',
    'tenure':'TempoContrato','PhoneService':'ServicoTelefone','MultipleLines':'MultiplasLinhas','InternetService':'ServicoInternet',
    'OnlineSecurity':'SegurancaOnline','OnlineBackup':'BackupOnline','DeviceProtection':'ProtecaoDispositivo','TechSupport':'SuporteTecnico',
    'StreamingTV':'TVStreaming','StreamingMovies':'FilmesStreaming','Contract':'TipoContrato','PaperlessBilling':'FaturaOnline',
    'PaymentMethod':'MetodoPagamento','Charges.Monthly':'FaturaMensal','Charges.Total':'FaturaTotal','Churn':'Evasao'
}
df = df.rename(columns={k:v for k,v in rename_map.items() if k in df.columns})

# Binárias Yes/No -> 1/0
for col in ['Parceiro','Dependentes','ServicoTelefone','MultiplasLinhas','SegurancaOnline','BackupOnline','ProtecaoDispositivo','SuporteTecnico','TVStreaming','FilmesStreaming','FaturaOnline','Evasao']:
    if col in df.columns:
        df[col] = df[col].map({'Yes':1,'No':0})

# Genero Female/Male -> 1/0
if 'Genero' in df.columns:
    df['Genero'] = df['Genero'].map({'Female':1,'Male':0})

# Traduções simples
if 'TipoContrato' in df.columns:
    df['TipoContrato'] = df['TipoContrato'].map({'Month-to-month':'Mensal','One year':'Anual','Two year':'Dois anos'})
if 'MetodoPagamento' in df.columns:
    df['MetodoPagamento'] = df['MetodoPagamento'].map({'Electronic check':'Cheque eletronico','Mailed check':'Cheque enviado','Bank transfer (automatic)':'Transferencia bancaria (automatica)','Credit card (automatic)':'Cartao de credito (automatico)'})
if 'ServicoInternet' in df.columns:
    df['ServicoInternet'] = df['ServicoInternet'].map({'DSL':'DSL','Fiber optic':'Fibra otica','No':'Sem servico de internet'})

df.head()


## 4) EDA rápida (visualizações principais)

In [ ]:
# Churn geral
ax = df['Evasao'].value_counts().plot.pie(autopct='%1.1f%%', startangle=90, colors=['skyblue','salmon'])
ax.set_ylabel('')
plt.title('Distribuicao da Evasao (Churn)')
plt.show()

print('Taxa de evasao geral: {:.2f}%'.format(df['Evasao'].mean()*100))

# Por Tipo de Contrato
if 'TipoContrato' in df.columns:
    sns.barplot(x='TipoContrato', y='Evasao', data=df, palette='viridis')
    plt.title('Taxa de Evasao por Tipo de Contrato')
    plt.ylabel('Taxa de Evasao')
    plt.xlabel('Tipo de Contrato')
    plt.show()

# Tempo de contrato e Fatura
for col in ['TempoContrato','FaturaMensal']:
    if col in df.columns:
        sns.histplot(data=df, x=col, hue='Evasao', kde=True, palette='coolwarm')
        plt.title(f'Distribuicao de {col} por Evasao')
        plt.show()


## 5) Correlacao (extra)

In [ ]:
# Contagem de servicos adicionais
serv_cols = ['SegurancaOnline','BackupOnline','ProtecaoDispositivo','SuporteTecnico','TVStreaming','FilmesStreaming','MultiplasLinhas']
serv_cols = [c for c in serv_cols if c in df.columns]
for c in serv_cols:
    df[c] = pd.to_numeric(df[c], errors='coerce').fillna(0).astype(int)
if serv_cols:
    df['NumServicosAdicionais'] = df[serv_cols].sum(axis=1)

num = df.select_dtypes(include=['number'])
corr = num.corr()
plt.figure(figsize=(12,9))
sns.heatmap(corr, cmap='coolwarm', center=0)
plt.title('Matriz de Correlacao')
plt.show()

if 'Evasao' in corr.columns:
    print('Correlacoes com Evasao (top):')
    print(corr['Evasao'].sort_values(ascending=False).head(10))


## 6) Conclusões (curtas)

- Maior risco: clientes com contrato Mensal, pouco tempo de casa e fatura mais alta.
- Incentivar planos de 1-2 anos e pagamentos automáticos.
- Onboarding forte nos 3 primeiros meses reduz churn.